In [13]:
from web3 import Web3
from eth_account import Account
import json

# ---------------- STEP 1: CREATE A WALLET ----------------
wallet = Account.create()
WALLET_ADDRESS = wallet.address
PRIVATE_KEY = wallet.key.hex()  # Store securely!

print(f"✅ Wallet Created!\nAddress: {WALLET_ADDRESS}\nPrivate Key: {PRIVATE_KEY}")

✅ Wallet Created!
Address: 0x31C014E5f9ad130c5Cf7a4526886c9AcffeBD1A3
Private Key: acdfd655510f3aedd17f38717d76db468e4aacb92a256722830435ebf75ee1b4


In [14]:

# ---------------- STEP 2: CONNECT TO GENSYN TESTNET ----------------
ALCHEMY_URL = "https://gensyn-testnet.g.alchemy.com/public"
web3 = Web3(Web3.HTTPProvider(ALCHEMY_URL))

if web3.is_connected():
    print("✅ Connected to Gensyn Testnet")
else:
    raise Exception("Failed to connect to Gensyn Testnet")


✅ Connected to Gensyn Testnet


In [25]:

# ---------------- STEP 3: CHECK BALANCE ----------------
balance = web3.eth.get_balance(WALLET_ADDRESS)
eth_balance = web3.from_wei(balance, "ether")
print(f"💰 Wallet Balance: {eth_balance} ETH")


💰 Wallet Balance: 0.011718599979570547 ETH


In [16]:
# ---------------- STEP 4: CONTRACT CONFIG ----------------

with open('../out/SwarmCoordinator.sol/SwarmCoordinator.json', 'r') as f:
   contract_abi = json.load(f)['abi']

contract_address = "0x6484a07281B72b8b541A86Ec055534223672c2fb"
contract = web3.eth.contract(address=contract_address, abi=contract_abi)


In [17]:
# ---------------- STEP 5: CURRENT ROUND, STAGE ----------------

# Get the current round and stage
current_round = contract.functions.currentRound().call()
current_stage = contract.functions.currentStage().call()

print(f"Current Round: {current_round}")
print(f"Current Stage: {current_stage}")

Current Round: 0
Current Stage: 0


In [ ]:
# ---------------- STEP 6: REGISTER PEER ----------------

# Peer ID and private key
peer_id = "grievepeer"

checksummed = Web3.to_checksum_address(WALLET_ADDRESS)
nonce = web3.eth.get_transaction_count(checksummed)


# Build the transaction
txn = contract.functions.registerPeer(peer_id).build_transaction({
    'chainId': 685685,  # Mainnet
    'gas': 21604,
    'gasPrice': web3.to_wei('50', 'gwei'),
    'nonce': nonce,
})

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(txn, private_key=PRIVATE_KEY)

# Send the transaction
tx_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)

# Print the transaction hash
print(f"Transaction hash: {web3.to_hex(tx_hash)}")

Transaction hash: 0x1d434db369d3f7a9aa1926fcaa0f5030065ce5ea4e67b1071b1a3dfdbb5f7fb5


In [ ]:
# ---------------- STEP 7: SUBMIT ANSWER ----------------

# wallet addr and nonce
checksummed = Web3.to_checksum_address(WALLET_ADDRESS)
nonce = web3.eth.get_transaction_count(checksummed)

payload = [

]

# Build the transaction
txn = contract.functions.submitWinners(
        0,
        ['grievepeer'],
    ).build_transaction({
    'chainId': 685685,  # Mainnet
    'gas': 10000000,
    'gasPrice': web3.to_wei('10', 'gwei'),
    'nonce': nonce,
})

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(txn, private_key=PRIVATE_KEY)

# Send the transaction
tx_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)

# Print the transaction hash
print(f"Transaction hash: {web3.to_hex(tx_hash)}")

Transaction hash: 0xdd5ec8e067150d3c949df9bd671e4b0aeac6ba3b7164366dc70f4490be34b555
